In [1]:
from __future__ import absolute_import
from __future__ import print_function
import numpy as np

import random
from keras.datasets import mnist
from keras.models import Model
from keras.layers import Input, Flatten, Dense, Dropout, Lambda
from keras.optimizers import RMSprop
from keras import backend as K

num_classes = 10
epochs = 20


def euclidean_distance(vects):
    x, y = vects
    sum_square = K.sum(K.square(x - y), axis=1, keepdims=True)
    return K.sqrt(K.maximum(sum_square, K.epsilon()))


def eucl_dist_output_shape(shapes):
    shape1, shape2 = shapes
    return (shape1[0], 1)

#définition de la contrastive_loss
def contrastive_loss(y_true, y_pred):
    margin = 1
    return 0.5 * y_true * y_pred + 0.5 * (1 - y_true) * (K.maximum(margin - y_pred, 0))

#création des pairs positives et négatives 
def create_pairs(x, digit_indices):
    pairs = []
    labels = []
    n = min([len(digit_indices[d]) for d in range(num_classes)]) - 1
    for d in range(num_classes):
        for i in range(n):
            z1, z2 = digit_indices[d][i], digit_indices[d][i + 1]
            pairs += [[x[z1], x[z2]]]
            inc = random.randrange(1, num_classes)
            dn = (d + inc) % num_classes
            z1, z2 = digit_indices[d][i], digit_indices[dn][i]
            pairs += [[x[z1], x[z2]]]
            labels += [1, 0]
    return np.array(pairs), np.array(labels)

#création du modèle
def create_network(input_shape):
    '''Base network to be shared (eq. to feature extraction).
    '''
    input = Input(shape=input_shape)
    x = Flatten()(input)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.1)(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.1)(x)
    x = Dense(256, activation='relu')(x)
    return Model(input, x)


def compute_accuracy(y_true, y_pred):
    pred = y_pred.ravel() < 0.5
    return np.mean(pred == y_true)

#définition de la métrique accuracy pour le réseau siamois
def accuracy(y_true, y_pred):
    return K.mean(K.equal(y_true, K.cast(y_pred < 0.5, y_true.dtype)))


# séparation du train et test 
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
input_shape = x_train.shape[1:]

# création des pairs sur le train et le test 
digit_indices = [np.where(y_train == i)[0] for i in range(num_classes)]
tr_pairs, tr_y = create_pairs(x_train, digit_indices)

digit_indices = [np.where(y_test == i)[0] for i in range(num_classes)]
te_pairs, te_y = create_pairs(x_test, digit_indices)

# network definition
network = create_network(input_shape)

input_a = Input(shape=input_shape)
input_b = Input(shape=input_shape)

# les poids seront partagés sur les deux branches du siamese
output_a = network(input_a)
output_b = network(input_b)

distance = Lambda(euclidean_distance,
                  output_shape=eucl_dist_output_shape)([output_a, output_b])

model = Model([input_a, input_b], distance)

# train
rms = RMSprop()
model.compile(loss=contrastive_loss, optimizer=rms, metrics=[accuracy])
model.fit([tr_pairs[:, 0], tr_pairs[:, 1]], tr_y,
          batch_size=128,
          epochs=epochs,
          validation_data=([te_pairs[:, 0], te_pairs[:, 1]], te_y))

# compute final accuracy on training and test sets
y_pred = model.predict([tr_pairs[:, 0], tr_pairs[:, 1]])
tr_acc = compute_accuracy(tr_y, y_pred)
y_pred = model.predict([te_pairs[:, 0], te_pairs[:, 1]])
te_acc = compute_accuracy(te_y, y_pred)

print('* Accuracy on training set: %0.2f%%' % (100 * tr_acc))
print('* Accuracy on test set: %0.2f%%' % (100 * te_acc))

Using TensorFlow backend.


11493376/11490434 [==============================] - 1s 0us/step




Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Train on 108400 samples, validate on 17820 samples
Epoch 1/20





108400/108400 [==============================] - 16s 148us/step - loss: 0.0932 - accuracy: 0.8907 - val_loss: 0.0503 - val_accuracy: 0.9368
Epoch 2/20
108400/108400 [==============================] - 15s 139us/step - loss: 0.0464 - accuracy: 0.9422 - val_loss: 0.0386 - val_accuracy: 0.9430
Epoch 3/20
108400/108400 [==============================] - 15s 139us/step - loss: 0.0359 - accuracy: 0.9505 - val_loss: 0.0351 - val_accuracy: 0.9484
Epoch 4/20
108400/108400 [==============================] - 15s 141us/step - loss: 0.0318 - accuracy: 0.9550 - val_loss: 0.0332 - val_accuracy: 0.9497
Epoch 5/20
108400/108400 [==============================] 